#### Table of Contents
[Problem Description and Discussion of Background](#problem)  
[Data Description](#data_descr)

# <a name="problem">Problem Description and Discussion of Background</a>
One of the major contributors to traffic delays are serious car accidents that require transporting of injured and/or clearing of wreckage. The business value we are attempting to provide is to answer the question "What is the best way to predict (and therefore avoid) serious car accidents when driving?" The goal of this exercise is to predict the severity of a given road collision when provided with certain details regarding the accident (e.g., weather and the road conditions) to allow drivers to drive more carefully or modify their travel plans in response to avoid encountering accidents.

# <a name="data_desc">Data Description</a>

To answer the above question, data will be needed regarding traffic accidents in a particular area. Such data will necessarily contain details about the accident, as well as an indication of whether or not the accident was serious. The data we are using for our analysis consists of tabular data of road collisions and their details from the Seattle, WA area collected between 01-01-2004 and 05-20-2020. The data includes location (lat, long, and address), type of collision, number involved (for persons, pedestrians, cyclists, and vehicles), incident date, whether the driver was inattentive, whether the driver was under the influence, and whether the driver was speeding. This data will be analyzed and formatted for the creation of a model. If necessary, additional features will be engineered for the creation of a successful model.

In [ ]:
import numpy as np
import pandas as pd

print('Hello Capstone Project Course!')


Hello Capstone Project Course!
